In [3]:
import pandas as pd
import json
import os
from collections import defaultdict

将 zh_msra_origin 转换成 w2ner 需要的格式, word 没用到, 就不构建了

In [4]:
input_dir = r"D:\code\github\W2NER\data\zh_msra_origin"
output_dir = r"D:\code\github\W2NER\data\zh_msra"
os.makedirs(output_dir, exist_ok=True)

In [10]:
def convert_file(input_file, output_file):
    # 真离谱, 这里有序列非常长的
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # 简单来说, 因为只有 3 个标签, 所以每 3 行是一个完整的数据
    # 但还是 dict 来保存数据好了
    result = defaultdict(list)
    for line in data:
        context = line["context"]
        entity_label = line["entity_label"]
        ner_list = []
        for item in line["span_position"]:
            start, end = map(int, item.split(";"))
            ner_list.append({"type": entity_label, "index": list(range(start, end+1))})
        result[context].extend(ner_list)
    
    # 排序下 ner_list
    data_list = []
    for sentence, ner_list in result.items():
        sentence = sentence.split()
        # 先跳过长度太长的, 模型不支持
        # 太长的序列, GPU 内存直接爆炸了, 再降点
        if len(sentence) > 124:
            continue
        ner_list = sorted(ner_list, key=lambda x:x["index"])
        data_list.append({"sentence": sentence, "ner": ner_list})
    print(len(data_list))

    # 生成文件
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=2)

In [11]:
for file in os.listdir(input_dir):
    input_file = os.path.join(input_dir, file)
    output_file = os.path.join(output_dir, file.split(".")[-1]+".json")
    convert_file(input_file, output_file)

4538
4241
41005
